In [16]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import load_model
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv2D
from keras.layers import Add, BatchNormalization, Activation
from sklearn.cluster import DBSCAN
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler

from CGA.cluster_filters import cluster_filters

In [17]:
model = load_model('../Models/NN/model_renet50.h5')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_2 (Dense)             (None, 10)                20490     
                                                                 
Total params: 23,608,202
Trainable params: 20,490
Non-trainable params: 23,587,712
_________________________________________________________________


In [18]:
resnet50_layers = model.layers[0].layers
for i, warstwa in enumerate(resnet50_layers):
    print(f'Warstwa ResNet50 {i + 1}: {warstwa.name}, Typ: {warstwa.__class__.__name__}, Shape: {warstwa.output_shape}')


Warstwa ResNet50 1: input_3, Typ: InputLayer, Shape: [(None, None, None, 3)]
Warstwa ResNet50 2: conv1_pad, Typ: ZeroPadding2D, Shape: (None, None, None, 3)
Warstwa ResNet50 3: conv1_conv, Typ: Conv2D, Shape: (None, None, None, 64)
Warstwa ResNet50 4: conv1_bn, Typ: BatchNormalization, Shape: (None, None, None, 64)
Warstwa ResNet50 5: conv1_relu, Typ: Activation, Shape: (None, None, None, 64)
Warstwa ResNet50 6: pool1_pad, Typ: ZeroPadding2D, Shape: (None, None, None, 64)
Warstwa ResNet50 7: pool1_pool, Typ: MaxPooling2D, Shape: (None, None, None, 64)
Warstwa ResNet50 8: conv2_block1_1_conv, Typ: Conv2D, Shape: (None, None, None, 64)
Warstwa ResNet50 9: conv2_block1_1_bn, Typ: BatchNormalization, Shape: (None, None, None, 64)
Warstwa ResNet50 10: conv2_block1_1_relu, Typ: Activation, Shape: (None, None, None, 64)
Warstwa ResNet50 11: conv2_block1_2_conv, Typ: Conv2D, Shape: (None, None, None, 64)
Warstwa ResNet50 12: conv2_block1_2_bn, Typ: BatchNormalization, Shape: (None, None, None,

In [19]:
resnet_model = model.get_layer('resnet50')

output_layer = 'conv4_block1_2_conv'

layer = resnet_model.get_layer(output_layer)
weights = layer.get_weights()[0]
biases = layer.get_weights()[1] 

In [20]:
weights.shape

(3, 3, 256, 256)

In [21]:
biases.shape

(256,)

In [58]:
def prune_filter(original_model, cut_off_layer_name, indexes):
    layer_to_prune = original_model.get_layer(cut_off_layer_name)
    all_layers = original_model.layers
    layer_to_prune_index = all_layers.index(layer_to_prune)

    # Find previous and next layers
    previous_layer = next((layer for layer in all_layers[:layer_to_prune_index][::-1] if isinstance(layer, Conv2D)), None)
    next_layer = next((layer for layer in all_layers[layer_to_prune_index + 1:] if isinstance(layer, Add)), None)
    
    # Models to slice the original model
    model_prev = Model(inputs=original_model.input, outputs=previous_layer.output)
    model_to_prune = Model(inputs=previous_layer.output, outputs=next_layer.output)
    model_next = Model(inputs=next_layer.output, outputs=original_model.output)
    
    combined_input = Input(shape=model_prev.input_shape[1:])
    output_prev = model_prev(combined_input)
    
    # Create pruned layer
    config = layer_to_prune.get_config()
    config['filters'] = len(indexes)
    
    new_layer = Conv2D(**config)
    input_shape = layer_to_prune.input_shape
    new_layer.build(input_shape)
    
    weights = layer_to_prune.get_weights()[0][:, :, :, indexes]
    if len(layer_to_prune.get_weights()) > 1:
        biases = layer_to_prune.get_weights()[1][indexes]
        new_weights = [weights, biases]
    else:
        new_weights = [weights]
    new_layer.set_weights(new_weights)
    
    x = output_prev
    for layer in model_to_prune.layers:
        if layer.name == cut_off_layer_name:
            x = new_layer(x)
            x = Conv2D(filters=256, kernel_size=(1, 1), name='adaptation_conv')(x)
        elif isinstance(layer, Conv2D):
            x = Conv2D(
                filters=layer.filters,
                kernel_size=layer.kernel_size,
                strides=layer.strides,
                padding=layer.padding,
                activation=None,
                use_bias=layer.use_bias,
                kernel_initializer=layer.kernel_initializer,
                bias_initializer=layer.bias_initializer
            )(x)
        elif isinstance(layer, BatchNormalization):
            x = BatchNormalization()(x)
        elif isinstance(layer, Activation):
            x = Activation(layer.activation)(x)
        elif isinstance(layer, Add):
            # Handling Add layer to connect branches
            skip_connection = x
            x = layer.input[1]
            x = Conv2D(
                filters=skip_connection.shape[-1],
                kernel_size=(1, 1),
                strides=(1, 1),
                padding='same',
                use_bias=False
            )(x)
            x = BatchNormalization()(x)
            x = Add()([skip_connection, x])

    output_next = model_next(x)
    pruned_model = Model(inputs=combined_input, outputs=output_next)
    print(pruned_model.summary())

    new_model = Sequential()
    new_model.add(model_prev)
    new_model.add(pruned_model)
    new_model.add(model_next)
    return new_model


In [23]:
weights.shape

(3, 3, 256, 256)

In [24]:
weights_list = weights.reshape(256, -1)
weights_list.shape

(256, 2304)

In [25]:
# from sklearn.cluster import KMeans
# 
# k = 100  # Liczba klastrów
# knn = KMeans(n_clusters=k)
# knn.fit(weights_list)
# klastry = knn.labels_
# 
# indexes = []
# for i in range(k):
#     indeksy_klastra = np.where(klastry == i)[0]
#     losowy_indeks = np.random.choice(indeksy_klastra)
#     indexes.append(losowy_indeks)
# 
# pruned_model = prune_filter(resnet_model, output_layer, indexes)

In [26]:
# dbscan = DBSCAN(eps=0.9, min_samples=1)
# dbscan.fit(weights_list)
# 
# 
# cluster_indices = np.unique(dbscan.labels_)
# 
# indexes = []
# for cluster_index in cluster_indices:
#     cluster_points = np.where(dbscan.labels_ == cluster_index)[0]
#     index = np.random.choice(cluster_points)
#     indexes.append(index)
#     
# pruned_model = prune_filter(resnet_model, output_layer, indexes)

In [27]:
def extract_feature_maps(model_, layer_name, input_data):
    intermediate_layer_model = Model(inputs=model_.input, outputs=model_.get_layer(layer_name).output)
    intermediate_output = intermediate_layer_model.predict(input_data)
    return intermediate_output

In [28]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_X = np.expand_dims(train_X, axis=-1)
train_X = np.repeat(train_X, 3, axis=-1)
train_X = np.pad(train_X, ((0, 0), (2, 2), (2, 2), (0, 0)), mode='constant')
train_y = to_categorical(train_y, 10)

In [30]:
feature_maps = extract_feature_maps(resnet_model, output_layer, np.random.rand(1, 224, 224, 3))

1/1 [==============================] - 1s 555ms/step


In [31]:
num_samples, height, width, num_filters = feature_maps.shape
feature_maps_flattened = feature_maps.reshape((num_samples, height * width, num_filters))

# Reshape to (num_samples * height * width, num_filters) for clustering
weights_list = feature_maps_flattened.reshape((num_filters,-1 ))

In [32]:
weights_list.shape

(256, 196)

In [42]:
from sklearn.decomposition import PCA
from CGA.cluster_CGA import cluster_algorithm
from CGA.CGA import complete_gradient_algorithm

pca = PCA(n_components=10)
data = pca.fit_transform(weights_list)

scaler = StandardScaler()
data = scaler.fit_transform(weights_list)
x, h, s = complete_gradient_algorithm(data)

x = scaler.fit_transform(x)
z = cluster_algorithm(x, h, s, data)

0
Max_d:  28.799713
Avg distances:  19.70332
Sigma:  1.6942488
x_d:  [15.79039877]
Cluster 1: [132, 4, 43, 68, 157, 171, 196, 107, 235, 29, 93, 221]
Cluster 2: [59, 123, 187, 251]
Cluster 3: [77, 13, 102, 141, 205, 38, 166, 230]
Cluster 4: [64, 0, 128, 192]
Cluster 5: [86, 22, 150, 214]
Cluster 6: [58, 122, 186, 250]
Cluster 7: [62, 126, 190, 254]
Cluster 8: [65, 1, 129, 193]
Cluster 9: [57, 121, 185, 249]
Cluster 10: [61, 125, 189, 253]
Cluster 11: [63, 191, 255, 127]
Cluster 12: [60, 51, 85, 124, 149, 188, 213, 252, 115, 154, 179, 243, 21, 46, 174, 76, 140, 204, 26, 90, 218, 110, 238, 12]
Cluster 13: [56, 120, 184, 248]
Cluster 14: [66, 2, 130, 194]
Cluster 15: [55, 119, 183, 247, 80, 94, 158, 208, 16, 144, 30, 222]
Cluster 16: [73, 9, 137, 201]
Cluster 17: [195, 3, 67, 131]
Cluster 18: [197, 5, 69, 133]
Cluster 19: [71, 7, 135, 199]
Cluster 20: [72, 200]
Cluster 21: [114, 50, 178, 242]
Cluster 22: [70, 6, 134, 198]
Cluster 23: [95, 31, 159, 223]
Cluster 24: [81, 209]
Cluster 25: [87

In [59]:
import random

indexes = [random.choice(l) for l in z]
pruned_model = prune_filter(resnet_model, output_layer, indexes)

ValueError: Found input tensor cannot be reached given provided output tensors. Please make sure the tensor KerasTensor(type_spec=TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='input_3'), name='input_3', description="created by layer 'input_3'") is included in the model inputs when building functional model.

In [37]:
prune_input = Input(shape=(32, 32, 3))

# for layer in model.layers[1:]:
#     layer.trainable = False

new_model = Sequential()
new_model.add(prune_input)
new_model.add(pruned_model)
# new_model.add(model.layers[1])
new_model.add(Dense(10, activation='softmax'))

In [38]:
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_model.fit(x = train_X, y= train_y, epochs=1)

1875/1875 [==============================] - 142s 73ms/step - loss: 0.3566 - accuracy: 0.8878


In [39]:
test_X = np.expand_dims(test_X, axis=-1)
test_X = np.repeat(test_X, 3, axis=-1)
test_X = np.pad(test_X, ((0, 0), (2, 2), (2, 2), (0, 0)), mode='constant')
result = new_model.evaluate(test_X, to_categorical(test_y, 10))

313/313 [==============================] - 24s 72ms/step - loss: 0.2121 - accuracy: 0.9333


In [40]:
new_model.save('../NN/pruned_model_mnist_renet50_first_proba_kde.h5')

In [41]:
model_final = load_model('../NN/pruned_model_mnist_renet50_first_proba_kde.h5')
result = model_final.evaluate(test_X, to_categorical(test_y, 10))
print(result)

313/313 [==============================] - 24s 73ms/step - loss: 0.2121 - accuracy: 0.9333
[0.21214155852794647, 0.9333000183105469]
